In [25]:
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np

In [26]:
f = pd.read_csv('final raw data.csv', na_filter=False)


In [27]:
data = f.drop(['Full count Pupae', 'Household_ID'], axis=1)

In [28]:
data.rename(columns={'Full count Larvae': 'class'}, inplace=True)
data.head(5)

,Hamlet,Container_ID,Container_position,Water_level,Lid,Fish,Meso,Micronecta,Source,Treatment,Larvae,class
0,0,1,0,25,1,0,0,0,0,0,1,206.7
1,0,2,1,25,1,0,0,0,1,0,0,0.0
2,0,5,0,25,0,0,0,0,1,0,0,0.0
3,0,6,0,25,1,0,0,0,0,1,0,0.0
4,0,5,1,30,1,0,0,0,0,1,0,0.0


In [29]:
pos_data = data[data['class'] >= 1]
neg_data = data[data['Larvae'] == 0]

print (pos_data.shape)
print (neg_data.shape)

(50, 12)
(500, 12)


In [30]:
pos_len = len(pos_data.index)
random_pos_data = pos_data.take(np.random.permutation(len(pos_data))[:pos_len])
samples = int(0.75 * pos_len)
train_pos_data = random_pos_data[:samples]
test_pos_data = random_pos_data[samples:]

In [31]:
final_pos_data = pd.concat([train_pos_data]*5)
times = len(final_pos_data)
final_neg_data = neg_data.take(np.random.permutation(len(neg_data))[:times])
frames = [final_pos_data, final_neg_data]
final_data = pd.concat(frames)
print (final_data.shape)

(370, 12)


In [32]:
final_data['class'] = final_data['class'].astype(int)
final_data.dtypes

Hamlet                int64
Container_ID          int64
Container_position    int64
Water_level           int64
Lid                   int64
Fish                  int64
Meso                  int64
Micronecta            int64
Source                int64
Treatment             int64
Larvae                int64
class                 int64
dtype: object

In [33]:
#final_data = final_data.fillna(-999)
#pd.isnull(final_data).any()

In [34]:
data_new = final_data.drop(['class'], axis=1)

In [35]:
data_class = final_data['class'].values

In [36]:
X_train, X_test, y_train, y_test = train_test_split(data_new, data_class,train_size=0.75, test_size=0.25)

In [37]:
tpot = TPOTRegressor(generations=5, population_size=20, num_cv_folds=5, random_state=42, verbosity=2)

Version 0.6.7 of tpot is outdated. Version 0.6.8 was released 2 days ago.


In [38]:
tpot.fit(data_new, data_class)

Optimization Progress:  17%|█▋        | 20/120 [00:31<02:43,  1.64s/pipeline]

Generation 1 - Current best internal CV score: 72.40371239234253


Optimization Progress:  32%|███▎      | 39/120 [01:42<05:48,  4.31s/pipeline]

Generation 2 - Current best internal CV score: 72.40371239234253


Optimization Progress:  48%|████▊     | 58/120 [03:09<05:56,  5.74s/pipeline]

...

Optimization Progress:  64%|██████▍   | 77/120 [04:01<01:54,  2.66s/pipeline]

Generation 4 - Current best internal CV score: 69.37934591891892


Optimization Progress:  82%|████████▎ | 99/120 [05:21<01:41,  4.85s/pipeline]

Generation 5 - Current best internal CV score: 69.37934591891892



Best pipeline: ExtraTreesRegressor(input_matrix, 24.0)


In [39]:
#print(tpot.score(X_test, y_test))

In [40]:
#tpot.export('tpot_exported_pipeline.py')

In [41]:
test_pos_data_X = test_pos_data.drop(['class'], axis=1)
test_pos_data_y = test_pos_data['class']
test_pos_data_X.shape
y_pred = tpot.predict(test_pos_data_X)
print (y_pred)


[  28.914   17.186   22.965   25.043  102.247   21.998   17.191   13.114
   29.954   10.68     9.       8.296   46.35 ]


In [42]:
from sklearn import metrics
np.sqrt(metrics.mean_squared_error(test_pos_data_y, y_pred))

57.928227069232967

In [43]:
##LINEAR REGRESSION

In [44]:
#feature selection with random forest
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier()
model.fit(data_new, data_class)

print(model.feature_importances_)
print (data_new.head(5))

[ 0.17402127  0.07035152  0.04357869  0.21730248  0.03641676  0.00044858
  0.03412891  0.01479275  0.02963516  0.04307979  0.33624411]
     Hamlet  Container_ID  Container_position  Water_level  Lid  Fish  Meso  \
381       7            12                   1           30    1     0     1   
405       8             5                   1            0    1     0     0   
518      10             5                   0           60    1     0     1   
331       6             5                   1           30    0     0     0   
56        1             5                   1           80    0     0     0   

     Micronecta  Source  Treatment  Larvae  
381           0       0          0       1  
405           0       0          1       1  
518           0       0          1       1  
331           0       0          0       1  
56            0       0          0       1  


In [45]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
model = LinearRegression()
rfe = RFE(model, 4)
fit = rfe.fit(data_new, data_class)
print (fit.n_features_)
print (fit.support_)
print (fit.ranking_)

4
[False False False False False False  True False  True  True  True]
[6 5 2 8 4 7 1 3 1 1 1]


In [46]:
#feature selection using chi square
import numpy
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(data_new, data_class)
numpy.set_printoptions(precision=3)
print(fit.scores_)

[  417.983    98.019    37.075  2106.898    66.185     6.      102.028
    56.938   171.422    96.371   185.   ]


In [54]:
#Using train/test split for feature selection
from sklearn.linear_model import LinearRegression
def train_test_rmse(X, y):
    linreg = LinearRegression()
    linreg.fit(X, y)
    y_pred = linreg.predict(X_test)
    return np.sqrt(metrics.mean_squared_error(test_pos_data_y, y_pred))

feature_cols = ['Hamlet', 'Water_level','Source']
X = data_new[feature_cols]
y = data_class
X_test = test_pos_data_X[feature_cols]
train_test_rmse(X, y)

52.684372094162882

In [48]:
feature_cols = ['Fish', 'Meso', 'Source']
X = data_new[feature_cols]
y = data_class
X_test = test_pos_data_X[feature_cols]
train_test_rmse(X, y)

51.322519667473486